In [1]:
#uses VectorAdd_#N(_noHR)(_S#noepsE_HRx#hrr)(_NIR)(_IR) - v1.1x

In [2]:
#changed namings
#inReadyC default to simplified definition

In [3]:
import sys
import math

In [4]:
#M>=1
M = input('Enter No Of Vectors (M): ')

Enter No Of Vectors (M): 10


In [5]:
#N>=1
N = input('Enter No Of Elements Per Vectors (N): ')

Enter No Of Elements Per Vectors (N): 11


In [6]:
#PR>=1
PR = input('Enter No Of Parallel Vectors (empty for 1): ')

Enter No Of Parallel Vectors (empty for 1): 12


In [7]:
N = int(N)
if N<1:
    N=1

M = int(M)
if M<1:
    M=1

if not PR:
    PR=1
else:
    PR = int(PR)
    if PR>M:
        PR = M

In [8]:
if N!=1:
    NOEPS = input('Enter No Of Vector Elements Per Series (>=1, <={}, empty for {}): '.format(N, N))

Enter No Of Vector Elements Per Series (>=1, <=11, empty for 11): 11


In [9]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH Default Value (empty for 10): ')

Enter IN_WIDTH Default Value (empty for 10): 16


In [10]:
Enable_Input_Registers = input('Enable Input Registers? (empty or 0 to disable, 1 to enable): ')

Enable Input Registers? (empty or 0 to disable, 1 to enable): 0


In [11]:
if N==1 or not NOEPS:
    NAdd=N
else:
    NAdd = int(NOEPS)
    if NAdd<1:
        NAdd=1
    elif NAdd>N:
        NAdd=N

HRR = math.ceil(N/NAdd);

if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if IN_WIDTH<1:
        IN_WIDTH=1

if not Enable_Input_Registers:
    Enable_Input_Registers=0
else:
    Enable_Input_Registers = int(Enable_Input_Registers)

In [12]:
ModuleName="MatrixAdd_{}_{}_{}PV_".format(M, N, PR)
if(HRR==1):
    ModuleName+="noHR_"
else:
    ModuleName+="S{}E_HRx{}_".format(NAdd, HRR)     
if Enable_Input_Registers<=0:
    ModuleName+="N"
ModuleName+="IR"

In [13]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    of = open("./"+ModuleName+".v", 'w+')

In [14]:
print("`timescale 1ns / 1ps\n", file=of)
print("module "+ModuleName, file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {}".format(IN_WIDTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
if HRR>1:
    print("output readyForNewVectorStart,", file=of)
lghrr=math.ceil(math.log2(HRR))
if lghrr==1:
    print("output vectorInSeries,", file=of)
elif lghrr>1:
    print("output [{}:0] vectorInSeries,".format(lghrr-1), file=of)
RS=math.ceil(M/PR)
lgRS=math.ceil(math.log2(RS))
if lgRS<=1:
    print("output reg vectorSetInNo = 0,", file=of)
else:
    print("output reg [{}:0] vectorSetInNo = 0,".format(lgRS-1), file=of)
print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for j in range(PR):
    for i in range(NAdd):
        print("A{}V{}, ".format(i, j), end='', file=of)
    print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for j in range(PR):
    for i in range(NAdd):
        print("B{}V{}, ".format(i, j), end='', file=of)
    print(file=of)
ar=RS*PR-M
ME = PR-ar-1
if ME==0:
    st1="V0"
else:
    st1="V0toV{}".format(ME)
print("output ", end='', file=of)
if HRR==1:
    print(st1, end='', file=of)
    print("outReady,", file=of)
    NE=NAdd
else:
    NE = N - NAdd*(HRR-1)
    while NE<=0:
        NE += NAdd
    if NE==1:
        print(st1, end='', file=of)
        print("S0outReady,", file=of)
    else:
        print(st1, end='', file=of)
        print("S0toS{}outReady,".format(NE-1), file=of)
    print("output ", end='', file=of)
    if NE==NAdd:
        print("VSNoutReady1, //not used", file=of)
    elif NE==NAdd-1:
        print(st1, end='', file=of)
        print("S{}outReady,".format(NE), file=of)
    else:
        print(st1, end='', file=of)
        print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
if ar==0:
    print("output reg VNoutReady1 = 0, //not used", file=of)
    print("output reg VNoutReady2 = 0, //not used", file=of)
else:
    if ME==(PR-2):
        st2="V{}".format(PR-1)
    else:
        st2="V{}toV{}".format(ME+1, PR-1)
    print("output ", end='', file=of)
    if HRR==1:
        print(st2, end='', file=of)
        print("outReady,", file=of)
    else:
        if NE==1:
            print(st2, end='', file=of)
            print("S0outReady,", file=of)
        else:
            print(st2, end='', file=of)
            print("S0toS{}outReady,".format(NE-1), file=of)
        print("output ", end='', file=of)
        if NE==NAdd:
            print("VSNoutReady2, //not used", file=of)
        elif NE==NAdd-1:
            print(st2, end='', file=of)
            print("S{}outReady,".format(NE), file=of)
        else:
            print(st2, end='', file=of)
            print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
if lghrr==1:
    print("output vectorOutSeries,", file=of)
elif lghrr>1:
    print("output [{}:0] vectorOutSeries,".format(lghrr-1), file=of)
if lgRS<=1:
    print("output reg vectorSetOutNo = 1,", file=of)
else:
    print("output reg [{}:0] vectorSetOutNo = {},".format(lgRS-1, RS-1), file=of)
print("output signed [IN_WIDTH:0] ", end='', file=of)
for j in range(PR):
    for i in range(NAdd):
        print("S{}V{}, ".format(i, j), end='', file=of)
    print(file=of)
print("output ", end='', file=of)
if HRR==1:
    print(st1, end='', file=of)
    print("earlyOutReady,", file=of)
else:
    if NE==1:
        print(st1, end='', file=of)
        print("S0earlyOutReady,", file=of)
    else:
        print(st1, end='', file=of)
        print("S0toS{}earlyOutReady,".format(NE-1), file=of)
    print("output ", end='', file=of)
    if NE==NAdd:
        print("CSNearlyOutReady1, //not used", file=of)
    elif NE==NAdd-1:
        print(st1, end='', file=of)
        print("S{}earlyOutReady,".format(NE), file=of)
    else:
        print(st1, end='', file=of)
        print("S{}toS{}earlyOutReady,".format(NE, NAdd-1), file=of)
if ar==0:
    print("output reg CNearlyOutReady1 = 0, //not used", file=of)
    print("output reg CNearlyOutReady2 = 0 //not used", file=of)
else:
    print("output ", end='', file=of)
    if HRR==1:
        print(st2, end='', file=of)
        print("earlyOutReady", file=of)
    else:
        if NE==1:
            print(st2, end='', file=of)
            print("S0earlyOutReady,", file=of)
        else:
            print(st2, end='', file=of)
            print("S0toS{}earlyOutReady,".format(NE-1), file=of)
        print("output ", end='', file=of)
        if NE==NAdd:
            print("CSNearlyOutReady2 //not used", file=of)
        elif NE==NAdd-1:
            print(st2, end='', file=of)
            print("S{}earlyOutReady".format(NE), file=of)
        else:
            print(st2, end='', file=of)
            print("S{}toS{}earlyOutReady".format(NE, NAdd-1), file=of)
print(");\n", file=of)

`timescale 1ns / 1ps

module MatrixAdd_10_11_10PV_noHR_NIR
#(
parameter IN_WIDTH = 16
)(
input clk, reset, enable,
output reg vectorSetInNo = 0,
input inReady,
input signed [IN_WIDTH-1:0] A0V0, A1V0, A2V0, A3V0, A4V0, A5V0, A6V0, A7V0, A8V0, A9V0, A10V0, 
A0V1, A1V1, A2V1, A3V1, A4V1, A5V1, A6V1, A7V1, A8V1, A9V1, A10V1, 
A0V2, A1V2, A2V2, A3V2, A4V2, A5V2, A6V2, A7V2, A8V2, A9V2, A10V2, 
A0V3, A1V3, A2V3, A3V3, A4V3, A5V3, A6V3, A7V3, A8V3, A9V3, A10V3, 
A0V4, A1V4, A2V4, A3V4, A4V4, A5V4, A6V4, A7V4, A8V4, A9V4, A10V4, 
A0V5, A1V5, A2V5, A3V5, A4V5, A5V5, A6V5, A7V5, A8V5, A9V5, A10V5, 
A0V6, A1V6, A2V6, A3V6, A4V6, A5V6, A6V6, A7V6, A8V6, A9V6, A10V6, 
A0V7, A1V7, A2V7, A3V7, A4V7, A5V7, A6V7, A7V7, A8V7, A9V7, A10V7, 
A0V8, A1V8, A2V8, A3V8, A4V8, A5V8, A6V8, A7V8, A8V8, A9V8, A10V8, 
A0V9, A1V9, A2V9, A3V9, A4V9, A5V9, A6V9, A7V9, A8V9, A9V9, A10V9, 
input signed [IN_WIDTH-1:0] B0V0, B1V0, B2V0, B3V0, B4V0, B5V0, B6V0, B7V0, B8V0, B9V0, B10V0, 
B0V1, B1V1, B2V1, B3V1, B4V1, B5V1, 

In [15]:
if RS!=1:
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\tvectorSetInNo <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(inReady) begin", file=of)
    if(HRR>1):
        print("\t\t\tif(vectorInSeries=={}) begin".format(HRR-1), file=of)
        print("\t", end='', file=of)
    print("\t\t\tif(vectorSetInNo=={}) begin".format(RS-1), file=of)
    if(HRR>1):
        print("\t", end='', file=of)
    print("\t\t\t\tvectorSetInNo <= 0;", file=of)
    if(HRR>1):
        print("\t", end='', file=of)
    print("\t\t\tend", file=of)
    if(HRR>1):
        print("\t", end='', file=of)
    print("\t\t\telse begin", file=of)
    if(HRR>1):
        print("\t", end='', file=of)
    if RS==2:
        print("\t\t\t\tvectorSetInNo <= 1;", file=of)    
    else:
        print("\t\t\t\tvectorSetInNo <= vectorSetInNo+1;", file=of)
    if(HRR>1):
        print("\t\t\t\tend", file=of)
    print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    print(file=of)

In [16]:
if ar>0:
    print("wire inReadyC = (vectorSetInNo!={}) & inReady;".format(RS-1), file=of)
    print(file=of)

In [17]:
for i in range (PR):
    if(HRR==1):
        if Enable_Input_Registers>0:
            print("VectorAdd_{}_noHR_IR#( .IN_WIDTH(IN_WIDTH) )".format(N), file=of)
        else:
            print("VectorAdd_{}_noHR_NIR#( .IN_WIDTH(IN_WIDTH) )".format(N), file=of)            
    else:
        if Enable_Input_Registers>0:
            print("VectorAdd_{}_S{}E_HRx{}_IR#( .IN_WIDTH(IN_WIDTH) )".format(N, NAdd, HRR), file=of)
        else:
            print("VectorAdd_{}_S{}E_HRx{}_NIR#( .IN_WIDTH(IN_WIDTH) )".format(N, NAdd, HRR), file=of)            
    print("\tVA_{}(clk, reset, enable,".format(i), file=of)
    if(HRR>1):
        if i==0:
            print("\treadyForNewVectorStart,", file=of)
            print("\tvectorInSeries,".format(i), file=of)
        else:
            print("\tLLreadyForNewDataSeries{}, //not used".format(i), file=of)
            print("\tLLinSeries{}, //not used".format(i), file=of)
    if(PR-i<=ar):
        print("\tinReadyC,", file=of)
    else:
        print("\tinReady,", file=of)
    print("\t", end='', file=of)
    for j in range(NAdd):
        print("A{}V{}, ".format(j, i), end='', file=of)
    print(file=of)
    print("\t", end='', file=of)
    for j in range(NAdd):
        print("B{}V{}, ".format(j, i), end='', file=of)
    print(file=of)    
    print("\t", end='', file=of)    
    if i==0:
        if HRR==1:
            print(st1, end='', file=of)
            print("outReady,", file=of)
        else:
            if NE==1:
                print(st1, end='', file=of)
                print("S0outReady, ", end='', file=of)
            else:
                print(st1, end='', file=of)
                print("S0toS{}outReady, ".format(NE-1), end='', file=of)
            if NE==NAdd:
                print("VSNoutReady1, //not used", file=of)
            elif NE==NAdd-1:
                print(st1, end='', file=of)
                print("S{}outReady,".format(NE), file=of)
            else:
                print(st1, end='', file=of)
                print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
    elif i==PR-ar:
        if HRR==1:
            print(st2, end='', file=of)
            print("outReady,", file=of)
        else:
            if NE==1:
                print(st2, end='', file=of)
                print("S0outReady, ", end='', file=of)
            else:
                print(st2, end='', file=of)
                print("S0toS{}outReady, ".format(NE-1), end='', file=of)
            if NE==NAdd:
                print("CSNoutReady2, //not used", file=of)
            elif NE==NAdd-1:
                print(st2, end='', file=of)
                print("S{}outReady,".format(NE), file=of)
            else:
                print(st2, end='', file=of)
                print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
    else:
        if HRR==1:
            print("VA{}OR, //not used".format(i), file=of)
        else:
            print("VA{}OR1, VA{}OR2, //both not used".format(i, i), file=of)
    if(HRR>1):
        if i==0:
            print("\tvectorOutSeries,".format(i), file=of)
        else:
            print("\tLLoutSeries{}, //not used".format(i), file=of)
    print("\t", end='', file=of)
    for j in range(NAdd):
        print("S{}V{}, ".format(j, i), end='', file=of)
    print(file=of)
    print("\t", end='', file=of)    
    if i==0:
        if HRR==1:
            print(st1, end='', file=of)
            print("earlyOutReady", file=of)
        else:
            if NE==1:
                print(st1, end='', file=of)
                print("S0earlyOutReady, ", end='', file=of)
            else:
                print(st1, end='', file=of)
                print("S0toS{}earlyOutReady, ".format(NE-1), end='', file=of)
            if NE==NAdd:
                print("VSNearlyOutReady1 //not used", file=of)
            elif NE==NAdd-1:
                print(st1, end='', file=of)
                print("S{}earlyOutReady".format(NE), file=of)
            else:
                print(st1, end='', file=of)
                print("S{}toS{}earlyOutReady".format(NE, NAdd-1), file=of)
    elif i==PR-ar:
        if HRR==1:
            print(st2, end='', file=of)
            print("earlyOutReady", file=of)
        else:
            if NE==1:
                print(st2, end='', file=of)
                print("S0earlyOutReady, ", end='', file=of)
            else:
                print(st2, end='', file=of)
                print("S0toS{}earlyOutReady, ".format(NE-1), end='', file=of)
            if NE==NAdd:
                print("VSNearlyOutReady2 //not used", file=of)
            elif NE==NAdd-1:
                print(st2, end='', file=of)
                print("S{}earlyOutReady".format(NE), file=of)
            else:
                print(st2, end='', file=of)
                print("S{}toS{}earlyOutReady".format(NE, NAdd-1), file=of)
    else:
        if HRR==1:
            print("VA{}EOR //not used".format(i), file=of)
        else:
            print("VA{}EOR1, VA{}EOR2 //both not used".format(i, i), file=of)
    print("\t);\n", file=of)

VectorAdd_11_noHR_NIR#( .IN_WIDTH(IN_WIDTH) )
	VA_0(clk, reset, enable,
	inReady,
	A0V0, A1V0, A2V0, A3V0, A4V0, A5V0, A6V0, A7V0, A8V0, A9V0, A10V0, 
	B0V0, B1V0, B2V0, B3V0, B4V0, B5V0, B6V0, B7V0, B8V0, B9V0, B10V0, 
	V0toV9outReady,
	S0V0, S1V0, S2V0, S3V0, S4V0, S5V0, S6V0, S7V0, S8V0, S9V0, S10V0, 
	V0toV9earlyOutReady
	);

VectorAdd_11_noHR_NIR#( .IN_WIDTH(IN_WIDTH) )
	VA_1(clk, reset, enable,
	inReady,
	A0V1, A1V1, A2V1, A3V1, A4V1, A5V1, A6V1, A7V1, A8V1, A9V1, A10V1, 
	B0V1, B1V1, B2V1, B3V1, B4V1, B5V1, B6V1, B7V1, B8V1, B9V1, B10V1, 
	VA1OR, //not used
	S0V1, S1V1, S2V1, S3V1, S4V1, S5V1, S6V1, S7V1, S8V1, S9V1, S10V1, 
	VA1EOR //not used
	);

VectorAdd_11_noHR_NIR#( .IN_WIDTH(IN_WIDTH) )
	VA_2(clk, reset, enable,
	inReady,
	A0V2, A1V2, A2V2, A3V2, A4V2, A5V2, A6V2, A7V2, A8V2, A9V2, A10V2, 
	B0V2, B1V2, B2V2, B3V2, B4V2, B5V2, B6V2, B7V2, B8V2, B9V2, B10V2, 
	VA2OR, //not used
	S0V2, S1V2, S2V2, S3V2, S4V2, S5V2, S6V2, S7V2, S8V2, S9V2, S10V2, 
	VA2EOR //not used
	);

Vect

In [18]:
if RS!=1:
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\tvectorSetOutNo <= {};".format(RS-1), file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(", end='', file=of)
    if HRR==1:
        print(st1, end='', file=of)
        print("earlyOutReady", end='', file=of)
    else:
        if NE==1:
            print(st1, end='', file=of)
            print("S0earlyOutReady", end='', file=of)
        else:
            print(st1, end='', file=of)
            print("S0toS{}earlyOutReady".format(NE-1), end='', file=of)
    print(") begin", file=of)
    if HRR!=1:
        print("\t\t\tif(vectorOutSeries=={}) begin".format(HRR-1), file=of)
        print("\t", end='', file=of)
    print("\t\t\tif(vectorSetOutNo=={}) begin".format(RS-1), file=of)
    if HRR!=1:
        print("\t", end='', file=of)
    print("\t\t\t\tvectorSetOutNo <= 0;", file=of)
    if HRR!=1:
        print("\t", end='', file=of)
    print("\t\t\tend", file=of)
    if HRR!=1:
        print("\t", end='', file=of)
    print("\t\t\telse begin", file=of)
    if HRR!=1:
        print("\t", end='', file=of)
    if RS==2:
        print("\t\t\t\tvectorSetOutNo <= 1;", file=of)
    else:
        print("\t\t\t\tvectorSetOutNo <= vectorSetOutNo+1;", file=of)
    if HRR!=1:
        print("\t", end='', file=of)
    print("\t\t\tend", file=of)
    if HRR!=1:
        print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    print(file=of)

In [19]:
print("endmodule", file=of)

endmodule


In [20]:
if __Print_To_File>0:
    of.close()